# Fase 2

### Integracion con datos de Spotify API
#### sources:
- https://medium.com/@eelopez088/data-analysis-with-spotify-api-a1507f48e9b0
- https://medium.com/@navodas88/using-spotify-api-for-data-collection-537ea2b4c0ab

#### Query di test Spotify

 DATAFRAME: top100_annual
 
 COLUMNA DE BUSQUEDA SPOTIFY: song_and_artist

In [1]:
import pandas as pd

# Cargo el csv
file_path = r"C:\Users\edo\Desktop\Final Project\billboard_weekly_ready.csv"

billboard_weekly_ready = pd.read_csv(file_path)
billboard_weekly_ready.rename(columns={"song_artist": "song_and_artist"}, inplace=True)
billboard_weekly_ready.head()


,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,song_and_artist
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021,Easy On Me - Adele
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021,Stay - The Kid LAROI & Justin Bieber
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021,Industry Baby - Lil Nas X & Jack Harlow
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021,Fancy Like - Walker Hayes
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021,Bad Habits - Ed Sheeran


In [14]:
billboard_weekly_ready.tail()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,song_and_artist
330082,1958-08-04,96,Over And Over,Thurston Harris,NaN,96,1,1958,Over And Over - Thurston Harris
330083,1958-08-04,97,I Believe In You,Robert & Johnny,NaN,97,1,1958,I Believe In You - Robert & Johnny
330084,1958-08-04,98,Little Serenade,The Ames Brothers,NaN,98,1,1958,Little Serenade - The Ames Brothers
330085,1958-08-04,99,I'll Get By (As Long As I Have You),Billy Williams,NaN,99,1,1958,I'll Get By (As Long As I Have You) - Billy Wi...
330086,1958-08-04,100,Judy,Frankie Vaughan,NaN,100,1,1958,Judy - Frankie Vaughan


### TEST Conexion a Spotify

In [ ]:
import spotipy

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# credenciales de la app
client_id = "INSERT_CLIENT_ID"
client_secret = "INSERT_CLIENT_SECRET"

# conexion a Spotify
spoti = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
))

print(f"Conexion con Spotify: OK")


Conexion con Spotify: OK


### TEST query track to spotify

In [ ]:

# cargo el dataset
file_path = r"C:\Users\edo\Desktop\Final Project\billboard_weekly_ready.csv"
billboard_weekly_ready = pd.read_csv(file_path)
billboard_weekly_ready.rename(columns={"song_artist": "song_and_artist"}, inplace=True)

# tomo la primera cancion del dataset
fila = billboard_weekly_ready.iloc[0]
consulta_cancion = fila["song_and_artist"]

# busqueda en Spotify
resultados = spoti.search(q=consulta_cancion, type="track", limit=1)

if resultados['tracks']['items']:
    track_info = resultados['tracks']['items'][0]
    artist_id = track_info["artists"][0].get("id")

 # intento obtener genero del artista
    try:
        artist_info = spoti.artist(artist_id)
        artist_genres = ", ".join(artist_info.get("genres", []))
    except Exception as e:
        artist_genres = None

    datos_cancion = {
        "- track_id": track_info.get("id"),
        "- title": track_info.get("name"),
        "- main_artist": track_info["artists"][0].get("name"),
        "- album": track_info["album"].get("name"),
        "- release_date": track_info["album"].get("release_date"),
        "- duration_ms": track_info.get("duration_ms"),
        "- explicit": track_info.get("explicit"),
        "- artist_genres": artist_genres
    }

    for k, v in datos_cancion.items():
        print(f"{k}: {v}")
else:
    print("No se encontró la canción:", consulta_cancion)



- track_id: 46IZ0fSY2mpAiktS3KOqds
- title: Easy On Me
- main_artist: Adele
- album: 30
- release_date: 2021-11-19
- duration_ms: 224694
- explicit: False
- artist_genres: soft pop


### TEST query track AUDIO FEATURES


In [69]:
# cancion de ejemplo
consulta_cancion = "Shape of You Ed Sheeran"

# busqueda en Spotify
resultados = spoti.search(q=consulta_cancion, type="track", limit=1)

if resultados['tracks']['items']:
    track_info = resultados['tracks']['items'][0]
    print(f'cancion encontrada: {track_info.get("name")} --> ID: {track_info.get("id")}')
    # intento con audio features
    try:
        audio_features = spoti.audio_features(track_info["id"])[0]
        print("audio features:", audio_features)
    except Exception as e:
        print("error download - audio features:", e)
else:
    print("cancion NO encontrada:", consulta_cancion)


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7qiZfU4dY1lWllzX7mPBI3 with Params: {} returned 403 due to None


cancion encontrada: Shape of You --> ID: 7qiZfU4dY1lWllzX7mPBI3
error download - audio features: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=7qiZfU4dY1lWllzX7mPBI3:
 None, reason: None


## !!!PROGRAMA PRINCIPAL!!!
### DATA INTEGRATION query track 

##### columnas:
- track_id
- track_name
- track_popularity
- album_name
- release_date
- duration_ms
- explicit
- artist_id
- artist_name

### LAST VERSION


#### Resumen de los cambios principales

- Evita duplicados usando una cache por song_artist.
- Limite de peticiones y varias claves: en caso de rate-limit o retry-after, cambia llaves    credenciales spotify
- Pausas adaptativa entre solicitudes
- Guarda 2 csv: datos validos y errores
- El programa continua donde se paro usando song_and_artist + date.
- Hora limite, se para automaticamente a las 08:00 del dia siguiente.

#### - link to API registration: https://developer.spotify.com/dashboard



In [ ]:
import os
import time
from datetime import datetime, timedelta

import pandas as pd
import requests

# credenciales API Spotify
# https://developer.spotify.com/dashboard
clients = [
    {"id": "INSERT_CLIENT_ID", "secret": "INSERT_CLIENT_SECRET", "name": "Key 1"},
    {"id": "INSERT_CLIENT_ID", "secret": "INSERT_CLIENT_SECRET", "name": "Key 2"},
    {"id": "INSERT_CLIENT_ID", "secret": "INSERT_CLIENT_SECRET", "name": "Key 3"},
]
# contiene el indice de las credenciales en uso [0,1,2]
current_client_id = 0


###########################################
# Funcion get_token - acceder a Spotify atravez de token 
###########################################

def get_token(client_id, client_secret):

    url= 'https://accounts.spotify.com/api/token'
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, data=data, auth=(client_id,client_secret))

    # en caso de error en la conexion Return None
    if response.status_code != 200:
        print(f"Error en obtener token de acceso: {response.status_code} --> {response.text}")
        return None
    
    # en caso OK, recupera el token de acceso 
    return response.json()["access_token"]

# Aplico la funcion con el current_clien_id
token = get_token(clients[current_client_id]['id'],clients[current_client_id]['secret'])
if not token:
    raise RuntimeError("❌ ERROR: imposible obtener token Spotify iniziale.")
print(f"🎉🎉 SUCCESS: Conexion a Spotify OK con clave {clients[current_client_id]['name']}")


###########################################
# Ruta files e configuracion input output
###########################################

SAVE_FOLDER = r"C:\Users\edo\Desktop\Final Project"
INPUT_FILE = f"{SAVE_FOLDER}/billboard_weekly_ready.csv"
OUTPUT_FILE = f"{SAVE_FOLDER}/spotify_weekly_integrated.csv"
ERROR_FILE = f"{SAVE_FOLDER}/spotify_weekly_errors.csv"


###########################################
# lectura files y setup dataframe
###########################################

# si no existe el file de input da ERROR
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"[ERROR] File de input no encontrado: {INPUT_FILE}")

df_input = pd.read_csv(INPUT_FILE)

# si no existe el file output lo crea
if not os.path.exists(OUTPUT_FILE):
    df_output = pd.DataFrame(columns=[
    'year', 'position', 'song', 'artist_main', 'song_and_artist', 'date',
    'track_id', 'track_name', 'track_popularity', 'album_name', 'release_date',
    'duration_ms', 'explicit', 'artist_id', 'artist_name', 'spotify_genres'
])
    df_output.to_csv(OUTPUT_FILE, index=False)
else:
    # si existe el file output lo pone come df_output
    df_output = pd.read_csv(OUTPUT_FILE)

# si no existe el file de errores lo crea
if not os.path.exists(ERROR_FILE):
    df_errors = pd.DataFrame(columns=["song_and_artist", "date", "reason"])
    df_errors.to_csv(ERROR_FILE, index=False)
else:
    # si existe el file error lo pone come df_errors
    df_errors = pd.read_csv(ERROR_FILE)

###########################################
# Inicializacion de processed
# Para saber que filas ya fueron procesadas en ejecuciones anteriores del codigo
###########################################
if os.path.exists(OUTPUT_FILE) and os.path.getsize(OUTPUT_FILE) > 0:
    # Creamos un set con clave unica "song_artist||date" para todas las filas ya guardadas
    processed = set(df_output["song_and_artist"] + "||" + df_output["date"].astype(str))
    print(f"Re-inicio desde la ejecución anterior — {len(processed)} filas ya procesadas.")
else:
    processed = set()
    print("- No hay progreso previo, inicio desde cero.")


###########################################
# Limite horario e cache
###########################################

# calculo la 8AM de dia siguente a cuando lanzo el programa
end_time = (datetime.now() + timedelta(days=1)).replace(hour=8, minute=0, second=0)

# guardo temporalmente los datos ya consultados de cada cancion
# evita hacer consultas duplicadas, si la misma canción aparece varias veces
spotify_cache = {}

# processed_count: contador de cuántas canciones se han procesado en esta sesión.
# para mostrar logs de progreso y para calcular retrasos adaptativos entre peticiones.
processed_count = 0


###########################################
# Funcion Query Spotify
###########################################

def query_spotify(song_artist):

    global current_client_id, token
    # headers es para generar el token de acceso a spotify
    headers = {"Authorization": f"Bearer {token}"}
    # params es lo que vamos a pedir a spotify:
    # "q": song_artist --> buscamos a partir de cancion y artista
    # "type": "track"  --> buscamos por canciones
    # "limit": 1       --> solo pillamos el primer resultado
    params = {"q": song_artist, "type": "track", "limit": 1}
    url = "https://api.spotify.com/v1/search"
    # lanzamos la query a spotify
    # resp va a contener tanto los datos de la cancion que errores en caso haya
    resp = requests.get(url, headers=headers, params=params)


    # si hay rate limit o token invalido, se lanza error
    # esto dos codigo en particular para manejar rate limit y invalid token
    if resp.status_code in [429, 401]:
        raise requests.HTTPError(f"HTTP {resp.status_code}", response=resp)

    # recoges todos los demas errores (4xx y 5xx)
    resp.raise_for_status()

    # extrae la respuesta de la API y la pone en un diccionario
    # la respuesta de spotify se esctrutura en {tracks} y [items]
    items = resp.json().get("tracks", {}).get("items", [])
    # return None si no hay resultados 
    if not items:  
        return None

    # del diccionario items tomamos:
    track = items[0]  # el diccionario relativo a la cancion
    artist_id = track["artists"][0]["id"]  # el id del artista principal
    # lanzamos query por las infos del artista
    artist_resp = requests.get(f"https://api.spotify.com/v1/artists/{artist_id}", headers=headers) 
    # recogemos los errores en caso haya
    artist_resp.raise_for_status()
    # artist_resp.json().get("genres", []) 
    # regresa un diccionario con la lista de genero del artista
    # ", ".join(...)
    # concat de todos los generos del artista
    genres = ", ".join(artist_resp.json().get("genres", [])) 

    # devolvemos un diccionario con toda la información de:
    # la canción y el artista que vamos a guardar en el CSV
    return {
        'track_id': track["id"],
        'track_name': track["name"],
        'track_popularity': track["popularity"],
        'album_name': track["album"]["name"],
        'release_date': track["album"]["release_date"],
        'duration_ms': track["duration_ms"],
        'explicit': track["explicit"],
        'artist_id': artist_id,
        'artist_name': track["artists"][0]["name"],
        'spotify_genres': genres
    }


###########################################
# Loop Principal - Procesamiento de canciones
###########################################

# Recorremos todas las filas del dataframe df
for idx, row in df_input.iterrows():
   
    # datetime.now() >= end_time --> si la hora actual supera el límite, se detiene el loop
    # Se para a las 8AM del día siguiente
    if datetime.now() >= end_time:
        print(f"Horario limite alcanzado: {end_time}. Interrupción del proceso.")
        break

    # Construimos una clave unica para cada canción+fecha: "cancion||fecha"
    # Para saber si ya procesamos esta fila anteriormente
    song_artist = row['song_artist']
    date_str = str(row['date'])
    key = f"{song_artist}||{date_str}"

    # Si song_artist no es un string valido o esta fila ya fue procesada (la clave existe en processed),
    # saltamos esta fila usando continue para no volver a consultarla ni guardarla de nuevo
    if not isinstance(song_artist, str) or key in processed:
        continue

    #### CACHE ####
    # Si ya tenemos datos de esta cancion en la cache temporal, los usamos directamente
    if song_artist in spotify_cache:
        data = spotify_cache[song_artist].copy()
        data['date'] = date_str  # actualizamos la fecha de la fila actual
        # Guardamos directamente en el CSV de salida sin volver a consultar Spotify
        pd.DataFrame([data]).to_csv(OUTPUT_FILE, mode='a', header=False, index=False)
        processed.add(key)  # agregamos la clave (cancion + artista + fecha) al set processed
        processed_count += 1  
        continue

    ###########################################
    # Query spotify con switch automatico de clave
    ###########################################
    success = False # para saber si la query ha sido exitosa
    attempts = 0    # recuento de los tentavios hecho con las llaves
    # intenta obtener info de Spotify hasta que:
    # success=True o hasta haber probado todas las llaves disponibles (attempts < len(clients))
    while not success and attempts < len(clients):
        try:
            # Consulta a Spotify para la cancion
            result = query_spotify(song_artist)
            if result:
                # Creamos un diccionario con toda la info de la fila: datos del CSV de entrada + resultados de Spotify
                row_data = {
                    'year': row['year'],
                    'position': row['rank'],
                    'song': row['song'],
                    'artist_main': row['artist'],
                    'song_and_artist': song_artist,
                    'date': date_str,
                    **result
                    # row_data = { ... , **result }
                    # unpacking del diccionario results y lo anyade a diccionario row_data {}
                }
                # Guardamos row_data en la cache para evitar consultas duplicadas
                spotify_cache[song_artist] = row_data.copy()
                # Guardamos row_data en el CSV de salida
                pd.DataFrame([row_data]).to_csv(OUTPUT_FILE, mode='a', header=False, index=False)
            else:
                # Si no hay resultados de Spotify, registramos el error en df_error
                pd.DataFrame([{'song_and_artist': song_artist, 'date': date_str, 'reason': 'Nessun risultato'}]).to_csv(
                    ERROR_FILE, mode='a', header=False, index=False
                )
            # Marcamos esta fila como procesada
            processed.add(key)
            processed_count += 1
            success = True

        ############    
        # Manejo de rate limit 
        ############
        except requests.HTTPError as e:
            # HTTPError captura el error generado de requests (los codigos 404, 429 etcetc)
            code = e.response.status_code if hasattr(e, 'response') else None
            # si e (el contenido del error) tiene el atributo respose hasattr(e, 'response')
            # entoces tomamos el codigo code (429, 404 etc)
            retry_after = int(e.response.headers.get("Retry-After", 0)) if hasattr(e, 'response') else 0
            # retry_after: 
            # Indica cuanto tiempo hay que esperar antes de hacer otra solicitud (util en caso de rate limit 429).
            # verifica si la respuesta HTTP contiene el header "Retry-After" y devuelve su valor en segundos.
            # Si no existe la respuesta o no hay el header, devuelve 0.
            

            #  HTTP == 429 → Spotify nos ha limitado las solicitudes (RATE LIMIT)
            # Convertimos retry_after en horas, minutos y segundos para mostrarlo de manera legible
            if code == 429:
                hours = retry_after // 3600
                minutes = (retry_after % 3600) // 60
                seconds = retry_after % 60
                formatted_time = f"{hours}h {minutes}m {seconds}s" if hours else f"{minutes}m {seconds}s"
                print(f"RATE LIMIT (429) — Spotify pide de esperar {formatted_time}")

            ############
            # Cambio automatico de clave/API key
            ############
            # Si hemos recibido un error (429 o 401), cambiamos inmediatamente a la siguiente clave disponible
            # usamos modulo para ciclar entre las claves
            current_client_id = (current_client_id + 1) % len(clients)
            # current_client_id es el indice de la clave
            # current_client_id + 1 → avanzamos a la siguente clave
            # % len(clients) → si llegamos al final de la lista, volvemos al índice 0            
            print(f"🔑 Pasaje a clave #{current_client_id+1}")
            # Obtenemos un nuevo token usando la clave actual (current_client_id)
            # Cada vez que cambiamos de clave, necesitamos un token nuevo
            token = get_token(clients[current_client_id]['id'], clients[current_client_id]['secret'])
            attempts += 1
            # Si ya hemos probado todas las claves disponibles
            if attempts >= len(clients):
                # Calculamos el tiempo de espera antes de reintentar
                # Si tenemos retry_after lo usamos +5s de margen, sino 60s por defecto
                wait_time = retry_after + 5 if retry_after else 60
                print(f"Todas las claves en rate-limited. Espero {wait_time//60}m {wait_time%60}s antes de re-empezar.")
                # Pausamos la ejecución por el tiempo calculado
                time.sleep(wait_time)
                # Reiniciamos el contador de intentos para volver a probar las claves
                attempts = 0
            continue

        except Exception as e:
            # Registramos cualquier otro error en el CSV de errores
            pd.DataFrame([{'song_and_artist': song_artist, 'date': date_str, 'reason': str(e)}]).to_csv(
                ERROR_FILE, mode='a', header=False, index=False
            )
            success = True

    
    # Cada 100 filas mostramos un mensaje de progreso.
    if processed_count % 100 == 0 and processed_count > 0:
        print(f"✅ {processed_count} filas procesadas — Ultimo artista: {song_artist}")

    # Calculamos un tiempo de espera dinamico entre requests para evitar ser bloqueados por rate limit
    # La formula es: 0.3 sec + (processed_count / total_filas) * 1.5
    # (processed_count / len(df_input)) * 1.5 -> aumenta progresivamente cuanto mas filas se procesan    
    adaptive_sleep = min(2.0, 0.3 + processed_count / len(df_input) * 1.5)
    time.sleep(adaptive_sleep)


print(f"ALL COMPLETE: Proceso completado, totales filas processadas: {processed_count}")


🎉🎉 SUCCESS: Conexion a Spotify OK con clave Key 1
Re-inicio desde la ejecución anterior — 330068 filas ya procesadas.
ALL COMPLETE: Proceso completado, totales filas processadas: 6


### CHECK AL FINAL DEL PROCESO

In [3]:
import pandas as pd

INPUT_FILE = r"C:\Users\edo\Desktop\Final Project\billboard_weekly_ready.csv"
OUTPUT_FILE = r"C:\Users\edo\Desktop\Final Project\spotify_weekly_integrated.csv"

# Corgo el csv
input_df = pd.read_csv(INPUT_FILE)
output_df = pd.read_csv(OUTPUT_FILE)


input_df.shape, input_df.isnull().sum()


((330087, 9),
 date                  0
 rank                  0
 song                  0
 artist                0
 last-week         32312
 peak-rank             0
 weeks-on-board        0
 year                  0
 song_artist           0
 dtype: int64)

In [ ]:

# Output shape e columnas
print("- Output shape:", output_df.shape)
print("\n- Columnas:", output_df.columns.tolist())

# null por columnas
output_df.isnull().sum()


- Output shape: (330070, 16)

- Columnas: ['year', 'position', 'song', 'artist_main', 'song_and_artist', 'date', 'track_id', 'track_name', 'track_popularity', 'album_name', 'release_date', 'duration_ms', 'explicit', 'artist_id', 'artist_name', 'spotify_genres']


year                     0
position                 0
song                     0
artist_main              0
song_and_artist          0
date                     0
track_id                 0
track_name               0
track_popularity         0
album_name               0
release_date             0
duration_ms              0
explicit                 0
artist_id                0
artist_name              0
spotify_genres      104058
dtype: int64

In [4]:

# Creiamo colonne chiave uniformi per il confronto
input_df['song_artist_date'] = input_df['song_artist'] + "___" + input_df['date'].astype(str)
output_df['song_artist_date'] = output_df['song_and_artist'] + "___" + output_df['date'].astype(str)

# Troviamo le righe presenti nell'input ma assenti nell'output
missing_rows = input_df[~input_df['song_artist_date'].isin(output_df['song_artist_date'])]

print(f"Filas que faltan en el output: {len(missing_rows)}")
missing_rows[['song_artist', 'date', 'rank']]


Filas que faltan en el output: 4


,song_artist,date,rank
121794,Party Ain't A Party - Queen Pen Featuring Tedd...,1998-07-11,95
121888,Party Ain't A Party - Queen Pen Featuring Tedd...,1998-07-04,89
121973,Party Ain't A Party - Queen Pen Featuring Tedd...,1998-06-27,74
122082,Party Ain't A Party - Queen Pen Featuring Tedd...,1998-06-20,83


In [ ]:
# cargo el csv error
err_df = pd.read_csv(ERROR_FILE)
err_df.head()

,song_and_artist,date,reason
